<a href="https://colab.research.google.com/github/shengy90/reinforcement-learning-an-introduction/blob/master/Chapter_3_Finite_Markov_Decision_Processes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**`Markov Decision Processess`** or MDPs:
- involve both evaluative feedback (like the bandit problems) and associative aspect (choosing different actions in different situations)
- classical formalisation of *sequential decision making*, taking into consideration both *immediate* and *delayed* rewards 
- we estimate the value $q_*(state, action)$ instead of just $q_*(action)$

**Things we'll be looking into:**
- returns 
- value functions 
- Bellman equations 

# 3.1 The Agent-Environment Interface

**Some nomenclature:**
- **`agent`** : the learner/ decision maker 
- **`environment`**: thing an `agent` interacts with. Basically everything in the system outside of the agent. 

![fig1](https://github.com/shengy90/reinforcement-learning-an-introduction/blob/master/misc/ch3fig1.png?raw=true)

**Agent and environment interact at each discrete timesteps:** $t = 0,1,2...$. At each timestep, the `agent` receives some representation of the environment's `state` $S_t$ and on that basis, selects an `action` $A_{t}$. Then, as a consequence of the action, the `agent` receives a `reward` $R_{t+1}$ and finds itself in a new state, $S_{t+1}$.


**The agent and the environment therefore give rise to a sequence** aka `trajectory`:

$$S_0, A_0, R_1, S_1, A_1, R_2, S_2, A_2....$$


**A finite MDP** (sets of states $\{S, A, R\}$ **all** have a finite number of elements. In this case, random variables $R_t$ and $S_t$ have well defined discrete probability distributions that depends **only on the preceding state and action**:

$$p(s',r | s,a) \dot = P \{S_t=s', R_t=r | S_{t-1}=s, A_{t-1}=a\},$$

for all $s', s \epsilon S$, $r \epsilon R$ and $a \epsilon A(s)$. 

The function $p$ defines the `dynamics` of the MDP. The dynamaics function $p : S X R X S X A \to [0,1]$ is an ordinary deterministic function of 4 arguments:

$$\sum_{s' \epsilon S} \sum_{r \epsilon R} p(s',r|s,a) = 1, \text{for all } s \epsilon S, a \epsilon A(s).$$

**This means:**
- probability of current state and rewards *only depends on* the immediate preceding state and action (Definition of 1st order Markov processes).


### **Example: A recycling robot 🤖**
[from Sutton and Barto: Reinforcement Learning An Introduction 2nd Edition, Chapter 3 example 3.3](http://incompleteideas.net/book/RLbook2020.pdf)

A mobile robot has the job of collecting empty soda cans in an office engironment. High-level decisions about how to search for cans are made by a RL agent *based on the current charge level of the battery* (aka the state):

- **The battery assumes 2 charge levels - high and low:** $S=\{\text{high, low}\}$
- **The agent can choose to search, wait or recharge:** $A=\{\text{search, wait, recharge}\}$
- **The agent receives the following rewards for the following situations:**
    - searching for a can depletes battery. if:
        - robot finds a soda can, *positive reward* $r_{search}$
        - battery becomes depleted, *large negative reward* $-3$
        - shuts down and wait to be rescued, *positive reward* $r_{wait}$
- **The robot has the following probabilities:**
    - $\alpha$: probability of robot having a high charge ending with a low charge after searching
    - $\beta$: probability of robot having low charge end ending up with a depleted charge

**Below shows the `transition graph` of this finite MDP process:**
![alt text](https://github.com/shengy90/reinforcement-learning-an-introduction/blob/master/misc/ch3fig2.png?raw=true)


# 3.2 Goals and Rewards

**Rewards:** a numerical value $R_{t} \epsilon \mathbb{R}$ that the `environment` passes to the `agent` 

**Goals:** maximise the rewards that the agent will receive *in the long run*

**Very often:**
- researches have also provided a reward of $-1$ for every timestep that passes to incentivise the agent finding the fastest possible solution. 
- also we need to give only appropriate rewards, i.e. make sure that when the agent maximises the reward, it also maximises our goals, not the 'sub goals'.
- e.g. in a chess game, if we reward the agent for every opponent's piece removed, the robot might be maximising the number of pieces removed and still lose the game (thereby achieving subgoals but not the main goal).
- reward signal is our way of communicating *what* we want achieved, not *how*!

# 3.3 Returns and Episodes

**`Expected return`** 

The sum of all future rewards the agent expects to get: $G_{t} \dot = \sum_{i=t}^{T} T_{i+1} $

**`Episodes`:** 

A *subsequence* e.g. plays of game, or tips through a maze, or any sort of repeated interaction. Each episode ends in a 'special state' called the `terminal state`, followed by a *reset* to a standard starting state (or to a sample from a standard distribution of starting states). The next episode begins independently of how the previous ended. Tasks with episodes are called `episodic tasks` where:
- $S$ : nonterminal states 
- $S^{+}$: terminal state 

**`Continuing tasks`:**

Noncontinuing tasks where time could go on to infinity. This makes the reward expression above problematic, because what we're trying to maximise is then 'infinite' i.e. impossible! To combat this, we can introduce a notion of `discounting`. 

**`Discounting`:**
$$G_{t} \dot = \sum_{k=0}^{\infty} \gamma^{k} R_{t+k+1},$$

Where $0 \leq \gamma \leq 1$ is called the `discount rate`. Gamma basically weights immediate rewards higher than some future distant reward. So getting a +1 in t=t+10 is worth more than at t=t+20. 

- $\gamma$ = 0 : agent is said to be *myopic* in the sense that it only cares about immediate reward
- $\gamma% = 1 : agent takes into account future rewards much more strongly 

Introducing the `discount rate` solves the infinity problem because as time tends to infinity:
$$G_{t} = \sum_{k=0}^{\infty} \gamma^k = \frac{1}{1-\gamma}$$

# 3.4 Unified Notation for Episodic and Continuing Tasks

To generalise notation so that we can include both episodic and continuing tasks, we'll define the reward function as:

$$G_t \dot = \sum_{k=t+1}^{T} \gamma^{k-t-1} R_{k},$$

Where $T=\infty$ (continuing tasks) or $\gamma=1$ (episodic tasks) but not both. 

# 3.5 Policies and Value Functions

#### **Definitions**

**`Value functions`**

Functions of states that estimate *how good* it is for the agent to be in a given state/ perform a given action in a given state. It measures the 'expected return' with respect to `policies`. 

**`Policy`**

A policy is a mapping from states to probabilities of selecting each possible actions. If an agent is following policy $\pi$ at time $t$, then $\pi(a|s)$ is the probability that $A_{t}=a$ if $S_{t}=s$.

Reinforcement learing methods specify how the agent's policy is changed as a result of its experience. 

The `value function` of a state $s$ under a policy $\pi$, denoted $v_{\pi}(s)$, is the `expected return` when starting in $s$ and following $\pi$ thereafter. For MDPs:

$$v_{\pi}(s) \space \dot \space = \space \mathbb{E}_{\pi}\big[G_t | S_t=s \big] = \mathbb{E}_{\pi} \Bigg[\sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \Bigg| S_t=s \Bigg], \text{ for all } s \epsilon S $$

We call $v_{\pi}$ the *state-value function for policy $\pi$*.


The value of taking action $a$ in state $s$ under a policy $\pi$ is therefore $q_\pi(s,a)$:

$$q_\pi(s,a) \space \dot = \space \mathbb{E}_\pi \big[ G_t | S_t=s,A_t=a \big] = \mathbb{E}_\pi \Bigg[\sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \Bigg| S_t=s, A_t=a \Bigg]$$.

We call $q_\pi(s,a)$ the *action-value function for policy $\pi$.

#### **Estimating Value Functions**

The value functions $v_\pi$ and $q_\pi$ can be estimated from experience:
- if an agent follows policy $\pi$ and maintains an average, for each state encountered, of the actual returns that have followed the state, then the average will converge to the state's value, $v_\pi(s)$. 
- if separate averages are kept for each action taken in each state, these averages will converge to the action values $q_\pi(s,a).$
- this is also called *Monte Carlo method* as it involves average over many random samples of actual returns. 
- this might not be practical for systems with many many states! for such systems, we need something more complex.

**One fundamental property of value functions in RL** is that they satisfy recursive relationships. Meaning for any policy $\pi$ and state $s$, the following consistency condition holds:

$$
\begin{align}
v_{\pi}(s) 
& \space \dot = \space \mathbb{E}_\pi [G_{t} | S_{t} = s] \\
&= \mathbb{E}_\pi [R_{t+1} + \gamma G_{t+1} | S_{t}=s] \\ 
&= \sum_a \pi(a|s) \sum_{s'}\sum_{r} p(s',r | s,a) \big[r + \gamma \mathbb{E}_\pi[G_{t+1}|S_{t+1}=s]\big] \\
&= \sum_a \pi(a|s) \sum_{s',r} p(s',r|s,a) \big[r + \gamma v_\pi(s') \big], \space \text{for all }s \epsilon S
\end{align}
$$

where $s'$ refers to the next state. This equation, is basically just the sum over all values of the three variables: $a$, $s'$ and $r$. 

For each triple, we compute its probability $\pi(a|s)p(s',r|s,a)$, weigh the quantity in brackets by that probability, then sum over all probabilities to get the expected value. 

This equation is also called the `Bellman equation` for $v_\pi$. It expresses the relationship between the value of a state and its successor states. 

![alt text](https://github.com/shengy90/reinforcement-learning-an-introduction/blob/master/misc/bellman.png?raw=true)


**What Bellman's equation is saying is basically:** the value of the current state should be equal to the discounted value of the expevted next state, plus any rewards expected along the way.

#### **Gridworld Example**

![gridworld](https://github.com/shengy90/reinforcement-learning-an-introduction/blob/master/misc/Screenshot%202020-05-06%20at%2015.49.15.png?raw=true)

Taken from [`Reinforcement Learning : An Introduction` by Sutton and Barto.](http://incompleteideas.net/book/RLbook2020.pdf)

**Left figure shows the retangular gridworld representation of a finite MDP. Each cell corresponds to the states of the `environment`.**
- There are 4 possible actions at each cell: move 1 grid *north*, *south*, *east* or *west*. 
- Actions that result in the agent leaving the grid has no effect, and returns -1 reward. 
- From State A, all 4 actions will move the agent to A' and get 10 points. 
- From state B, all 4 actions will move the agent to B' and get 5 points.
- All other actions get 0 points. 

**Right figure shows $v_\pi$, the state-value function for the equiprobable random policy with discount rate of $\gamma$ = 0.9.**
- Notice negative values near the lower edge - these are results of the high probability of hitting the edge under this policy
- State A has the highest value under this 'equiprobable policy'. Note that it's value is < 10, because as a result of landing in state A, the agent would end up having a high chance of falling off the grid.
- State B on the other hand, is valued more than its immediate reward of 5 as ending up in B' is still a relatively safe spot which *might* allow the user to end up in A or B again and getting the additional points. 


**TO:DO** CODE AND SOLVE THIS PROBLEM.

# 3.6 Optimal Policies and Optimal Value Functions

Value functions can be use to rank different policies, i.e. if policy $\pi$ is equal to or better than $\pi'$, then its expected return is also greater than or equal to that of $\pi'$ for all states. In short: $\pi \geq \pi'$ if and only if $v_{\pi}(s) \geq v_{\pi'}(s)$ for all $s\epsilon S. 

The *optimal policy*  (denoted as $\pi_*$) is the policy that's better than or equal to all other policies. Its state-value function, is defined as:
$$v_*(s) \dot = \max_\pi v_\pi(s) \space \text{ for all } s \epsilon S.$$


**Note that**  because $v_*$ is the value function for a poilcy, it must also satisfy the self-consistent condition giben by the Bellman equation. And because this is the optimal value function, it could be written in a special form without reference to any specific policy:


$$
\begin{align}
v_*(s) 
&= \max_{a \epsilon A(s)} q_{\pi_{*}}(s,a) \\
&=\max_{a} \mathbb{E}_{\pi_*} \big[G_t | S_t=s, A_t=a \big] \\
&=\max_{a} \mathbb{E}_{\pi_*} \big[R_{t+1} + \gamma G_{t+1} | S_t=s, A_t=a \big] \\
&=\max_{a} \mathbb{E} \big[ R_{t+1} + \gamma v_* (S_{t+1}) | S_t=s, A_t=a \big] \\ 
&=\max_{a} \sum_{s',r} p(s',r|s,a)[r + \gamma v_*(s')]
\end{align}
$$

The Bellman optimality equation for $q_*$ is:

$$
\begin{align}
q_*(s,a) 
&= \mathbb{E} \big[ R_{t+1} + \gamma \max_{a'} q_*(S_{t+1}, a') | S_t=s, A_t=a \big] \\
&= \sum_{s',r} p(s',r|s,a) [r+ \gamma \max_{a'} q_*(S',
\end{align}
$$


### **Optimal solution to gridworld**

![alt text](https://github.com/shengy90/reinforcement-learning-an-introduction/blob/master/misc/Screenshot%202020-05-06%20at%2016.42.13.png?raw=true)

The figures above show the optimal solution to gridworld example. $v_*$ shows the state value of every given state, whilst $\pi_*$ shows the corresponding optimal policies where multiple arrows mean all those arrow have the equal action-values.

### **Optimal solution to recycling robot**

**Recall that for the recycling robot problem:**
- 2 states: high (h), low(l)
- 3 actions : search (s), wait (w) and recharge(re)

Since there are only 2 states, Bellman optimality equation therefore consists only of 2 equations: $v_*(h)$ and $v_*(l)$.

#### **Equation for $v_*(h)$ (optimal value function for high battery levels**


**Recall that in high battery state, the policy is *do not recharge***.
- Action 1: search with s'=(high or low)
- Action 2: wait with s'=(high or low)

1. **Action 1: Seach with s'=high** :=> $v(h,s) = p(h|h,s)[r(h,s,h) + \gamma v_*(h)$
2. **Action 1: Search with s'=low** :=> $v(l,s) = p(h|h,s)[r(h,s,l) + \gamma v_*(l)$
3. **Action 2: Wait with s'=high** :=> $v(h,w) = p(h|h,w)[r(h,s,h) + \gamma v_*(h)$
4. **Action 2: Wait with s'=low**  :=> $v(l,w) = p(h|h,w)[r(h,s,l) + \gamma v_*(l)$

Therefore:

$$
\begin{align}
v_*(h) 
&= \max\bigg(v(h,s)+v(l,s), v(h,w)+v(l,w) \bigg) \\
&= \max\bigg(
    \alpha[r_s + \gamma v_*(h) + (1-\alpha)[r_s + \gamma v_*(l)], 
    1[r_w+\gamma v_*(h)] + 0[r_w + \gamma v_*(l)] \bigg) \\
&= \max\bigg(r_s + \gamma[\alpha v_*(h) + (1-\alpha) v_*(l)],
    r_w + \gamma v_*(h)\bigg)
\end{align}
$$

#### **Equation for $v_*(l)$ (optimal value function for low battery levels**

**Recall that in low battery state, the policy is *do not search***.
Applying the same approach as above, we get:

$$
v_*(l) = max\bigg(\\
\beta r_s -3(1-\beta) + \gamma[1-\beta)v_*(h) + \beta v_*(l)], \\
r_w \gamma v_*(l), \\
\gamma v_*(h) \\
\bigg)
$$

# 3.7 Optimality and Approximation

- Although it's possible to solve for the value function for the optimal policy theoretically, it's often not possible (without high computational cost) in the real world
- Therefore in the next few chapters, we'll look at different numerical approximation that allows us to approximate the optimal policies.